### https://towardsdatascience.com/how-to-forecast-sales-with-python-using-sarima-model-ba600992fa7d

In [1]:
import warnings
import itertools
import numpy as np
from numpy import concatenate, savetxt, unique, array, subtract
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import merge, DataFrame, Series
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib
from math import sqrt
from statistics import mean


In [2]:
df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True,
                 squeeze=True, dayfirst=True, engine='python')
df1a = pd.DataFrame(index =['Month'], columns=['prediction', 'actual'])
pred_date = '2019-03-01'

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df.columns =['MeterNo', 'Month', 'kWh']
df_DDH = pd.read_csv('DDH_Monthly.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
df_DDH['Date'] = pd.to_datetime(df_DDH['Date'], dayfirst=True)
df_DDH.columns =['Month', 'DDH']
df = merge(df, df_DDH, on='Month')#left_index=True, right_index=True)
df_meters = pd.read_csv('CHL-Meters.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
df = merge(df, df_meters, on='MeterNo')
y = df.set_index('Month')
y.drop(y.columns[[0]], axis = 1, inplace = True)
y = y.sort_index()

ya = y.groupby(y.index)['kWh'].mean()
yb = y.groupby(y.index)['DDH'].mean()
yc = y.groupby(y.index)['Area_m2'].mean()
z = merge(ya, yb, left_index=True, right_index=True)
z = merge(z, yc, left_index=True, right_index=True)
print(z)


                   kWh  DDH    Area_m2
Month                                 
2013-01-01  475.450704  318  72.517606
2013-02-01  468.250000  311  72.517606
2013-03-01  455.806338  380  72.517606
2013-04-01  390.042254  254  72.517606
2013-05-01  280.778169  168  72.517606
...                ...  ...        ...
2020-10-01  343.144366  183  72.517606
2020-11-01  425.767606  220  72.517606
2020-12-01  554.538732  326  72.517606
2021-01-01  560.669014  358  72.517606
2021-02-01  515.232394  261  72.517606

[98 rows x 3 columns]


In [3]:
endog = z.iloc[:,0]
exog = z.iloc[:,1:]
mod = sm.tsa.statespace.SARIMAX(endog = endog,
                            order=(1, 1, 1),
                            seasonal_order=(0, 1, 1, 12),
                            exog = exog,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
results = mod.fit()
pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
y_forecasted = pred.predicted_mean
y_forecasted =y_forecasted.to_frame()
y_truth = y[pred_date:]
df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
df1.drop(df1.columns[[2,3]], axis = 1, inplace = True)
df1.columns =['prediction', 'actual']
df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
print(df1)

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


            prediction  actual    error
2019-03-01  388.365029     586  -197.63
2019-03-01  388.365029     492  -103.63
2019-03-01  388.365029     630  -241.63
2019-03-01  388.365029     327    61.37
2019-03-01  388.365029    1156  -767.63
...                ...     ...      ...
2021-02-01  503.679723     234   269.68
2021-02-01  503.679723     486    17.68
2021-02-01  503.679723     513    -9.32
2021-02-01  503.679723     430    73.68
2021-02-01  503.679723    1719 -1215.32

[6816 rows x 3 columns]


In [4]:
# Each meter per month
df2 = df1.dropna()
print(df2)

            prediction  actual    error
2019-03-01  388.365029     586  -197.63
2019-03-01  388.365029     492  -103.63
2019-03-01  388.365029     630  -241.63
2019-03-01  388.365029     327    61.37
2019-03-01  388.365029    1156  -767.63
...                ...     ...      ...
2021-02-01  503.679723     234   269.68
2021-02-01  503.679723     486    17.68
2021-02-01  503.679723     513    -9.32
2021-02-01  503.679723     430    73.68
2021-02-01  503.679723    1719 -1215.32

[6816 rows x 3 columns]


In [5]:
mse = mean_squared_error(df2['actual'],df2['prediction'])
print(mse)

50290.750835059465


In [6]:
rmse = np.sqrt(mse)
print(rmse)

224.2559939779971


In [7]:
# Summary of each meter for each month
def Summary(x):
    return round(Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()]),2)
df2.apply(Summary)

,prediction,actual,error
min,199.36,-1821.00,-1595.65
max,589.73,2152.00,2109.81
mean,365.52,368.06,-2.54
sum,2491401.71,2508683.00,-17278.80
count,6816.00,6816.00,6816.00


In [8]:
#Mean Square Error Calc
def mse(g):
    mse = mean_squared_error(g['actual'], g['prediction'])
    return Series({'mse':mse})

df2a = df2.groupby(df2.index)['prediction'].sum()
df2b = df2.groupby(df2.index)['actual'].sum()
df2c = df2.groupby(df2.index).apply(mse).reset_index()
df2c = df2c.set_index('index')
df3 = merge(df2a, df2b, left_index=True, right_index=True)
df3 = merge(df3, df2c, left_index=True, right_index=True)
df3['rmse'] = round(np.sqrt(df3['mse']),2)
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['%error'] = (df3['error']/df3['actual'])*100
df3.columns =['sum of predictions', 'sum of actuals', 'mse', 'rmse', 'error', '%error']
print(df3)

            sum of predictions  sum of actuals            mse    rmse  \
2019-03-01       110295.668186          122566   60104.155312  245.16   
2019-04-01       100434.297420          109520   52906.052462  230.01   
2019-05-01        89360.347433           86028   27338.290362  165.34   
2019-06-01        68276.908000           64519   16086.896989  126.83   
2019-07-01        57534.737886           60027   12975.079814  113.91   
2019-08-01        56617.582678           55343   11865.783327  108.93   
2019-09-01        66815.087563           73711   19650.699605  140.18   
2019-10-01        89334.711923           91436   33335.263087  182.58   
2019-11-01       124643.455650          119083   52846.073572  229.88   
2019-12-01       149377.004881          151669   89952.280756  299.92   
2020-01-01       158004.588594          157278   97963.299531  312.99   
2020-02-01       148228.266357          152810   98365.885032  313.63   
2020-03-01       140504.224129          131657   59

In [9]:
df3.apply(Summary)

,sum of predictions,sum of actuals,mse,rmse,error,%error
min,56617.58,55343.00,11865.78,108.93,-15431.94,-15.84
max,167483.31,159230.00,107668.55,328.13,8847.22,6.72
mean,103808.40,104528.46,50290.75,211.68,-720.05,-0.73
sum,2491401.71,2508683.00,1206978.02,5080.22,-17281.28,-17.58
count,24.00,24.00,24.00,24.00,24.00,24.00


for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False, period=1)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

for m in meter_ids:
    df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.columns =['MeterNo', 'WeekNo', 'kWh']
    y = df.set_index(['WeekNo'])
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    #print(y)
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
            except: 
                continue

for m in meter_ids:
    df_meters = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, 
                            squeeze=True, dayfirst=True, engine='python')
    df_meters['Date'] = pd.to_datetime(df_meters['Date'], dayfirst=True)
    df_meters.columns =['MeterNo', 'Month', 'kWh']
    y = df_meters.set_index('Month')
    df_DDH = pd.read_csv('DDH_Monthly.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df_DDH['Date'] = pd.to_datetime(df_DDH['Date'], dayfirst=True)
    df_DDH.columns =['Month', 'DDH']
    z = df_DDH.set_index('Month')
    y = merge(y, z, left_index=True, right_index=True)
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    endog = y.iloc[:,0]
    exog = y.iloc[:,1:]
    freq = 'MS' # month start
    mod = sm.tsa.statespace.SARIMAX(endog = endog,
                                order=(1, 1, 1),
                                seasonal_order=(0, 1, 1, 12),
                                exog = exog,
                                freq = freq,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit()
    pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
    y_forecasted = pred.predicted_mean
    y_forecasted =y_forecasted.to_frame()
    y_truth = y[pred_date:]
    df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
    df1.drop(df1.columns[[2]], axis = 1, inplace = True)
    df1.columns =['prediction', 'actual']
    df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
    df1a = df1a.append(df1)


p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))